In [54]:
import pandas as pd
import numpy as np
from scipy import stats

In [34]:
df = pd.read_csv('Data')
df.drop(['Unnamed: 0'], axis = 1, inplace = True)
df.head()

,Марка,Модель,Год,Город,Пробег(км),Объем двигателя (см³),Мощность двигателя (л.с.),Цилиндров,Коробка,Привод,Цвет,Цена(руб),Форма двигателя
0,ABM,ABM Jazz 125,2015,Вязьма,7400,125,11.0,1.0,МКПП-4,цепь,чёрный,36000,рядное
1,ABM,ABM ZR 200,2013,Егорьевск,15000,200,19.0,1.0,МКПП-5,цепь,чёрный,63000,v-образное
2,ABM,ABM ZR 200,2014,Сиверский,1500,200,15.0,1.0,МКПП-5,цепь,зелёный,90000,v-образное
3,ABM,ABM Alpha 110,2023,Татарская Каргала,700,110,9.0,1.0,МКПП-4,цепь,чёрный,62000,рядное
4,ABM,ABM Alpha 110,2023,Фрязино,500,110,7.0,1.0,МКПП-4,цепь,чёрный,75000,рядное


***Гипотеза 1:***   \
H0 Механика надежнее других коробок передач на 20% (уровень значимости установим в 5%)\
Для того, чтобы проверить гипотезу, нам необходимо понять, что мы будет сравнивать. Так как информации о поломках у нас нет, а из всех данных только пробег и год могут говорить о времени эксплуатации/износе автомобиля, будем опираться на эти показатели в предположении, что все представленные мотоциклы на текущий момент исправны. Также заметим, что механика была внедрена в технику намного раньше, поэтому год в данном случае не несет большой ценности. Итого, мы будем проверять гипотезу о том, что средний пробег на механике выше, чем средний пробег на других коробках передачи:\
$X$ - средний пробег на механике, $Y$ - средний пробег на других коробках передач\
Тогда $H_{0}: \mu_{X} = 1.2 \mu_{Y},\\
H_{1}: \mu_{X} \not= 1.2 \mu_{Y}\$

$$
z_{obs} = \frac{\bar X - 1.2\bar Y}{\sqrt{\frac{s_x^2}{n_x} + \frac{1.44s_y^2}{n_y}}};
$$

In [85]:
def Mech_Tr(line):
    """Функция определяет, является ли коробка передач механической"""
    if 'МКПП' in line:
        return(True)
    else:
        return(False)

In [87]:
mask = df['Коробка'].apply(Mech_Tr) # Маска для отбора строк с механической коробкой передач

mech = df[mask]['Пробег(км)'] # Пробег на механике
other = df[~mask]['Пробег(км)'] # Пробег на других коробках передач

mu_x = mech.mean() 
var_x = mech.var() # Необходимые данные для механики
n_x = len(mech)

mu_y = other.mean()
var_y = other.var() # Необходимые данные для других коробок передач
n_y = len(other)

In [84]:
alpha = 0.05 # Уровень значимости
k = 1.2 # Коэффициент, показывающий надежность механики относительго других коробок передач

z_obs = (mu_x - k*mu_y)/np.sqrt(var_x/n_x + k**2*var_y/n_y)
z_crit = stats.norm.ppf(1 - alpha/2)

print(f'{z_obs:.3} попадает в ({-z_crit:.3} , {z_crit:.3}) => Нулевая гипотеза НЕ отвергается')

1.07 попадает в (-1.96 , 1.96) => Нулевая гипотеза НЕ отвергается


***Гипотеза 2:***   \
H0: Мотоциклы, произведенные с 2018, мощнее на 10% по сравнению с мотоциклами, выпущенными до 2003 (Уровень значимости 1%)

Показателем мощности в нашем случае будет мощность двигателя:\
$X$ - мощность мотоциклов, выпущенных с 2018, $Y$ - мощность мотоциклов, выпущенных до 2003\
Тогда $H_{0}: \mu_{X} = 1.1 \mu_{Y},\\
H_{1}: \mu_{X} \not= 1.1 \mu_{Y}\$

$$
z_{obs} = \frac{\bar X - 1.1\bar Y}{\sqrt{\frac{s_x^2}{n_x} + \frac{1.21s_y^2}{n_y}}};
$$

In [144]:
modern = df[df['Год'] >= 2018]['Мощность двигателя (л.с.)'] # Мощность современных мотоциклов
dated = df[df['Год'] < 2003]['Мощность двигателя (л.с.)'] # Мощность более старых мотоциклов

mu_x = modern.mean() 
var_x = modern.var() # Необходимые данные для механики
n_x = len(modern)

mu_y = dated.mean()
var_y = dated.var() # Необходимые данные для других коробок передач
n_y = len(dated)

In [145]:
alpha = 0.01 # Уровень значимости
k = 1.1 # Коэффициент, показывающий превосходство по мощности

z_obs = (mu_x - k*mu_y)/np.sqrt(var_x/n_x + k**2*var_y/n_y)
z_crit = stats.norm.ppf(1 - alpha/2)

print(f'{z_obs:.3} попадает в ({-z_crit:.3} , {z_crit:.3}) => Нулевая гипотеза НЕ отвергается')

2.07 попадает в (-2.58 , 2.58) => Нулевая гипотеза НЕ отвергается


***Гипотеза 3:***   \
H0: Мотоциклисты покупают свои транспортные средства в неклассических цветах чаще (уровень значимости 5%)

Классическими цветами будет считать черный и белый:\
Для того, чтобы ответ был положительным, достаточно, чтобы для какого-нибудь $i > 0.5$ гипотеза не отвергалась
Для проверки данное гипотезы, предположим, что выборочные дисперсии классических и неклассических цветов в генеральной совокупности равны. Проверять гипотезу будет с использованием t-теста, проведя бинаризацию цветов: 1 - неклассические, 0 - классические.Тогда\
H0: $\mu = i$,\
H1: $\mu \not= i$

In [212]:
alpha = 0.05
color = (df['Цвет'] != 'чёрный') & (df['Цвет'] != 'белый')
color = color.astype('int64')
for i in np.linspace(0.5, 1, 50):
    mu = i
    if stats.ttest_1samp(color, mu).pvalue > alpha:
        print(f'Нулевая гипотеза НЕ отвергается при i = {round(i, 2)}')
        break

Нулевая гипотеза НЕ отвергается при i = 0.56
